## Projeto
Classificação binária - Real ou Deepfake.

---

Link do desafio no Kaggle:
https://www.kaggle.com/competitions/image-classification-real-or-ai-generated-photo/data

In [ ]:
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import random

### Open and investigate Dataset

In [ ]:
from pathlib import Path
notebook_path = Path().resolve()

dataset_base_path = notebook_path.parent / "dataset/"

train_path = dataset_base_path / "train/train"
test_path = dataset_base_path / "test/test"

train_images = sorted(os.listdir(train_path), key=lambda x: int(os.path.splitext(x)[0]))
test_images = sorted(os.listdir(test_path), key=lambda x: int(os.path.splitext(x)[0]))

train_labels = pd.read_csv(dataset_base_path / "train.csv")
test_labels = pd.read_csv(dataset_base_path / "test.csv")

In [ ]:
def print_images(images, label, total_images = 6):
    sample_images = random.sample(images, total_images)
    
    fig, axes = plt.subplots(1, total_images, figsize=(12, 2))
    fig.suptitle(label, fontsize=16)
    for ax, img_name in zip(axes.flatten(), sample_images):
        img = Image.open(train_path / img_name)
        ax.imshow(img)
        ax.set_title(img_name)
        ax.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
label_real_images = 1
label_ai_generated = 0

real_train_images = train_labels[train_labels['Label'] == label_real_images]['Image'].tolist()
fake_train_images = train_labels[train_labels['Label'] == label_ai_generated]['Image'].tolist()

print_images(images=real_train_images, label="Real")
print_images(images=fake_train_images, label="Ai-Gen")

In [ ]:
print("Check class balacing")
print(train_labels['Label'].value_counts().to_dict())

print()
print(f"Train samples: {len(train_images)}")
print(f"Test samples: {len(test_images)}")

print()
print("Sample of Labels Table")
print(train_labels.head())

In [ ]:
plt.figure(figsize=(4, 3))
train_labels['Label'].value_counts().plot(kind='bar', color=['green', 'red'])
plt.title("Distribution of Classes in Train Set")
plt.xlabel("Label")
plt.ylabel("Number of Images")
plt.xticks([0,1], ['Real (0)', 'AI-Generated (1)'], rotation=0)
plt.tight_layout()
plt.show()

Como mostrado no gráfico acima, as classes estão balanceadas no conjunto de teste, isto facilita a análise e evita chances de viés de classe.

#### Pre-processamento

In [ ]:
from sklearn.model_selection import train_test_split

labels_dict = dict(zip(train_labels['Image'], train_labels['Label']))

train_targets = [labels_dict[f] for f in train_images]

X_train_img, X_val_img, y_train, y_val = train_test_split(
    train_images, train_targets, test_size=0.2, stratify=train_targets, random_state=42
)

print(X_train_img[:5], y_train[:5])
print(X_val_img[:5], y_val[:5])

In [ ]:
import numpy as np

In [ ]:
def load_images(image_list, base_path, target_size=(128, 128)):
    data = []
    for i, img_name in enumerate(image_list):
        if i % 100 == 0:
            print(f"Loading image {i+1} of {len(image_list)}: {img_name}")
        img_path = base_path / img_name
        img = Image.open(img_path).convert("RGB").resize(target_size)
        data.append(np.array(img) / 255.0)
    return np.array(data)

print(f"Loading training images...")
X_train = load_images(X_train_img, train_path)
print(f"Loading validation images...")
X_val = load_images(X_val_img, train_path)
print(f"Loading test images...")
X_test = load_images(test_images, test_path)

y_train = np.array(y_train)
y_val = np.array(y_val)

print("Train set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)
print("Test set:", X_test.shape)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Random Forest trabalha em 2D: [amostras, features]
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_val_flat = X_val.reshape(X_val.shape[0], -1)

print("Training Random Forest...")
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train_flat, y_train)

y_pred = rf_model.predict(X_val_flat)

In [ ]:
print("\nValidation results:")
print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred, digits=3))
print(f"Accuracy: {accuracy_score(y_val, y_pred):.3f}")

In [ ]:
# (pré-processamento, normalização, validação, modelagem, comparação).

In [ ]:
# Random forest ?
# SVN ?
# CNN